In [1]:
### import required libraries

import json

In [2]:
res_list = [{'app_success': "we couldn't find any revelant clients from the nyt news."},
            {'app_success': "we couldn't find any revelant clients from the wsj news."},
            {'app_success': "we couldn't find any revelant clients from the reuters news."},
            {'app_success': "we couldn't find any revelant clients from the reuters news."}]


In [6]:
res_list = [{'app_success': "we couldn't find any revelant clients from the nyt news."},
            {'app_success': "we couldn't find any revelant clients from the wsj news."},
            {'app_success': "we couldn't find any revelant clients from the reuters news."},
            {'app_success': "success", 
            'records': [{'company':'jepg', 'broker': "xxx", "broker_email": "xxx@cbre.com", "news": ["jepg acquired xxx", "jepg acquired xxx2"]}]}]

In [10]:
res_list = [{'app_success': "we couldn't find any revelant clients from the nyt news."},
            {'app_success': "success", 'records': [{'company':'jepg', 'broker': "xxx", "broker_email": "xxx@cbre.com", "news": ["jepg acquired xxx3"]}, 
                                                   {'company':'apple', 'broker': "xxx", "broker_email": "xxx@cbre.com", "news": ["apple acquired xxx3"]}]},
            {'app_success': "we couldn't find any revelant clients from the reuters news."},
            {'app_success': "success", 
            'records': [{'company':'jepg', 'broker': "xxx", "broker_email": "xxx@cbre.com", "news": ["jepg acquired xxx", "jepg acquired xxx2"]}]}]

In [5]:
a = []
b =", ".join(a)
if b == '':
    print(1)

1


In [11]:
slack_dict = {}
slack_dict['content'] = []

for each_web_result in res_list:
    try:
        for each_record in each_web_result['records']:
            if slack_dict['content']:
                if each_record['company'] in [i['company'] for i in slack_dict['content']]:
                    for i in slack_dict['content']:
                        if each_record['company'] == i['company'] and each_record['news'] not in i['news']:
                            i['news'].extend(each_record['news'])
                else:
                    slack_dict['content'].append(each_record)
            else:
                slack_dict['content'].append(each_record)
    except:
        pass

if slack_dict['content']:
    slack_dict['message'] = 'Here are the important m&a news for the past 1 week.'
else:
    slack_dict['message'] = 'No important m&a news found for the past 1 week.'
    del slack_dict['content']

In [12]:
def convert_to_slack_block(slack_dict):

    message = {
        "blocks": [
                    {
                        "type": "section",
                        "text": {
                            "type": "plain_text",
                            "text": slack_dict['message']
                        }
                    }
                ]
            }

    if 'content' in slack_dict.keys():

        content_list = slack_dict['content']
        
        for each_content in content_list:
            message['blocks'].append({'type':'divider'})
            one_block = list(each_content.values())

            company_info = one_block[:3]
            company_info[0] = "*" + company_info[0] + "*"
            news = one_block[3]

            num_space = (len(company_info) - len(news))
            num_space_list = [' '] * num_space

            if num_space>0:
                news.extend(num_space_list)
            elif num_space<0:
                company_info.extend(num_space_list)
            
            fields = []
            for i in range(len(company_info)):
                fields.append({"type":"mrkdwn", "text": company_info[i]})
                fields.append({"type":"mrkdwn", "text": news[i]})
            
            message['blocks'].append({"type": "section", "fields":fields})

    return message

In [13]:
convert_to_slack_block(slack_dict)

{'blocks': [{'type': 'section',
   'text': {'type': 'plain_text',
    'text': 'Here are the important m&a news for the past 1 week.'}},
  {'type': 'divider'},
  {'type': 'section',
   'fields': [{'type': 'mrkdwn', 'text': '*jepg*'},
    {'type': 'mrkdwn', 'text': 'jepg acquired xxx3'},
    {'type': 'mrkdwn', 'text': 'xxx'},
    {'type': 'mrkdwn', 'text': 'jepg acquired xxx'},
    {'type': 'mrkdwn', 'text': 'xxx@cbre.com'},
    {'type': 'mrkdwn', 'text': 'jepg acquired xxx2'}]},
  {'type': 'divider'},
  {'type': 'section',
   'fields': [{'type': 'mrkdwn', 'text': '*apple*'},
    {'type': 'mrkdwn', 'text': 'apple acquired xxx3'},
    {'type': 'mrkdwn', 'text': 'xxx'},
    {'type': 'mrkdwn', 'text': ' '},
    {'type': 'mrkdwn', 'text': 'xxx@cbre.com'},
    {'type': 'mrkdwn', 'text': ' '}]}]}